## -- Notebook: 02_silver_layer.ipynb
-- Camada: Silver
-- Objetivo: Limpar, enriquecer e padronizar dados da camada Bronze.

## -- 1. Criar a Tabela Delta na Camada Silver
-- Inicialmente, vamos criar a tabela Silver com uma carga completa dos dados limpos e enriquecidos.
-- Usaremos uma CTE para organizar a lógica de transformação.

In [0]:
%sql
CREATE TABLE IF NOT EXISTS mc_labs.trips_silver
USING DELTA
AS
WITH
 -- CTE para enriquecer os dados da Bronze com nomes de cidades
 enriched_trips AS (
     SELECT
         tr.tpep_pickup_datetime AS pickup_datetime,
         tr.tpep_dropoff_datetime AS dropoff_datetime,
         tr.trip_distance,
         tr.fare_amount,
         tr.pickup_zip,
         tr.dropoff_zip,
         pz.city_name AS pickup_city_name,
         dz.city_name AS dropoff_city_name,
         tr.ingestion_timestamp
     FROM
         mc_labs.trips_bronze AS tr
     LEFT JOIN
         mc_labs.zip_code_info AS pz ON tr.pickup_zip = pz.zip
     LEFT JOIN
         mc_labs.zip_code_info AS dz ON tr.dropoff_zip = dz.zip
 )
SELECT
 pickup_datetime,
 dropoff_datetime,
 CAST(pickup_datetime AS DATE) AS pickup_date, -- Adiciona uma coluna de data para facilitar análises
 trip_distance,
 fare_amount,
 pickup_zip,
 dropoff_zip,
 pickup_city_name,
 dropoff_city_name,
 ingestion_timestamp
FROM
 enriched_trips
WHERE
 pickup_city_name IS NOT NULL -- Filtra viagens com cidades de origem válidas
 AND dropoff_city_name IS NOT NULL -- Filtra viagens com cidades de destino válidas
 AND trip_distance > 0 -- Garante distância positiva
 AND fare_amount > 0; -- Garante valor de tarifa positivo

In [0]:
%sql
-- 2. Verificar o Esquema e os Dados da Tabela Silver
DESCRIBE DETAIL mc_labs.trips_silver;

In [0]:
%sql
DESCRIBE HISTORY mc_labs.trips_silver;

In [0]:
%sql
SELECT * FROM mc_labs.trips_silver LIMIT 10;

## -- 3. Simular uma Ingestão Incremental na Camada Silver usando MERGE INTO
-- Esta é a forma mais comum e eficiente de atualizar a camada Silver.
-- Vamos supor que novos dados chegaram na Bronze (que você inseriu no notebook anterior)
-- e queremos processá-los para a Silver.

## -- Primeiro, vamos criar uma CTE que representa os NOVOS dados a serem processados da Bronze
-- (poderíamos filtrar por ingestion_timestamp > último_timestamp_processado)
-- Para este exemplo, vamos pegar todos os dados da Bronze que ainda não estão na Silver
-- (ou que foram atualizados na Bronze, se tivéssemos uma lógica de atualização lá).
-- Uma abordagem mais robusta filtraria por `ingestion_timestamp` ou por um ID único.
-- Para simplificar, vamos processar os registros da Bronze que não estão na Silver,
-- ou que tiveram o `fare_amount` alterado (simulando uma atualização na Bronze).

In [0]:
%sql
WITH
 -- CTE para identificar os novos/atualizados dados da Bronze
 new_or_updated_bronze_data AS (
     SELECT
         tr.tpep_pickup_datetime AS pickup_datetime,
         tr.tpep_dropoff_datetime AS dropoff_datetime,
         tr.trip_distance,
         tr.fare_amount,
         tr.pickup_zip,
         tr.dropoff_zip,
         pz.city_name AS pickup_city_name,
         dz.city_name AS dropoff_city_name,
         tr.ingestion_timestamp
     FROM
         mc_labs.trips_bronze AS tr
     LEFT JOIN
         mc_labs.zip_code_info AS pz ON tr.pickup_zip = pz.zip
     LEFT JOIN
         mc_labs.zip_code_info AS dz ON tr.dropoff_zip = dz.zip
     WHERE
         pz.city_name IS NOT NULL -- Filtra viagens com cidades de origem válidas
         AND dz.city_name IS NOT NULL -- Filtra viagens com cidades de destino válidas
         AND tr.trip_distance > 0 -- Garante distância positiva
         AND tr.fare_amount > 0 -- Garante valor de tarifa positivo
         -- No cenário real, adicionaríamos um filtro aqui para pegar apenas os dados
         -- que foram ingeridos na Bronze desde a última execução da Silver.
         -- Ex: AND tr.ingestion_timestamp > (SELECT MAX(ingestion_timestamp) FROM mc_labs.trips_silver)
 )
MERGE INTO mc_labs.trips_silver AS target
USING new_or_updated_bronze_data AS source
ON target.pickup_datetime = source.pickup_datetime
AND target.pickup_zip = source.pickup_zip
AND target.dropoff_zip = source.dropoff_zip
WHEN MATCHED AND target.fare_amount <> source.fare_amount THEN -- Exemplo: se o fare_amount mudou, atualiza
 UPDATE SET
     target.dropoff_datetime = source.dropoff_datetime,
     target.trip_distance = source.trip_distance,
     target.fare_amount = source.fare_amount,
     target.pickup_city_name = source.pickup_city_name,
     target.dropoff_city_name = source.dropoff_city_name,
     target.ingestion_timestamp = source.ingestion_timestamp -- Atualiza o timestamp de ingestão
WHEN NOT MATCHED THEN
 INSERT (pickup_datetime, dropoff_datetime, pickup_date, trip_distance, fare_amount, pickup_zip, dropoff_zip, pickup_city_name, dropoff_city_name, ingestion_timestamp)
 VALUES (source.pickup_datetime, source.dropoff_datetime, CAST(source.pickup_datetime AS DATE), source.trip_distance, source.fare_amount, source.pickup_zip, source.dropoff_zip, source.pickup_city_name, source.dropoff_city_name, source.ingestion_timestamp);

## -- 4. Verificar os resultados do MERGE e o histórico da Silver

In [0]:
%sql
SELECT * FROM mc_labs.trips_silver WHERE pickup_zip IN (10001, 10003) ORDER BY pickup_datetime;

In [0]:
%sql
DESCRIBE HISTORY mc_labs.trips_silver; -- Observe uma nova versão no histórico